In [2]:
pip install langchain langchain-community openai faiss-cpu transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install unstructured

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install chromadb

In [22]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
import pandas as pd


os.environ["OPENAI_API_KEY"] = 'sk-proj-oQQ7gfDv5ErI9p1otbb5T3BlbkFJQUFrRjKOyjfCnTj8gYdt'


data_loader = CSVLoader("./Health_Data.csv")
cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter(
    separator="\n"
)

docs = data_loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

In [23]:
map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 AI 헬스 트레이너 FITPLE 입니다. 당신이 갖고 있는 정보를 참고하여 고객에게 알맞는 운동을 운동 종목(준비 운동/근력 운동/유산소 운동) / 운동 이름 : / 운동 세트수 / 추천해준 운동의 효과 설명) 형식으로 계획을 만들어서 추천해주세요. : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_chain = map_prompt | ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

def map_docs(inputs):
    documents, question = inputs["documents"], inputs["question"]
    return "\n\n".join(
        map_chain.invoke({"context": doc.page_content, "question": question}).content
        for doc in documents
    )

map_results = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            주어진 문장들을 이용해 최종 답변을 작성해주세요. 그리고 항상 똑같은 운동만 추천해주지말고 여러 운동을 추천해주세요. 만약 주어진 문장들 내에 답변을 위한 내용이 포함되어있지 않다면, 답변을 꾸며내지 말고, 모른다고 답해주세요.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

reduce_chain = {"context": map_results, "question": RunnablePassthrough()} | reduce_prompt | ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])


In [24]:
response = reduce_chain.invoke("나는 키 174에 몸무게 75kg, 30살 남자야 오늘 운동 일정을 대신 계획해줘")

print(response.content)

키 174cm에 몸무게 75kg인 30살 남성을 위한 오늘의 운동 일정을 준비했습니다. 

운동 종목: 인치웜 (inchworm)
운동 세트수: 4 세트
추천해준 운동의 효과 설명: 인치웜은 복근을 강화하고 허리를 잡아주는데 도움을 줍니다. 몸 전체를 움직이면서 유연성과 근력을 향상시킬 수 있는 운동입니다.

이 운동을 꾸준히 실시하면 체지방을 감소시키고 근력을 키우는 데 도움을 줄 것입니다. 꾸준한 노력과 올바른 영양섭취를 함께 실천해주시길 바라며, 신체 상태에 주의하면서 운동을 즐기시기 바랍니다. 건강한 신체를 위한 운동 활동을 응원합니다! 🏋️‍♂️💪


In [74]:
pip install Flask

In [75]:
pip install Flask langchain langchain-community openai faiss-cpu transformers unstructured tiktoken chromadb

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install flask-cors

  Obtaining dependency information for flask-cors from https://files.pythonhosted.org/packages/8b/52/2aa6285f104616f73ee1ad7905a16b2b35af0143034ad0cf7b64bcba715c/Flask_Cors-4.0.1-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [25]:
from flask import Flask, jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.before_request
def before_request_func():
    cache_dir = "./.cache/"
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        os.makedirs(cache_dir)

@app.route('/api/recommendation', methods=['POST'])  # 엔드포인트 이름을 변경합니다
def get_recommendation():
    data = request.json
    question = data.get("question")
    response = reduce_chain.invoke(question)
    return jsonify({"response": response.content})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)
    

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.47:5000
Press CTRL+C to quit
172.30.1.47 - - [28/May/2024 14:49:04] "POST /api/recommendation HTTP/1.1" 200 -
